# Get All Accounts

In [1]:
import os
import boto3
import json
import pandas as pd
import ipywidgets
import jupyter
from time import time
from datetime import datetime, timedelta
from tqdm.notebook import tqdm

In [2]:
awsProfiles = boto3.Session(profile_name = 'training').available_profiles

In [3]:
class AWSQueryStartDateTooOld(Exception): pass

def validateStartDate(StartTime:datetime = 0):
    # Compare the StartDate value against the validation rules. 
    # Raise an error if it is not valid
    if StartTime < datetime.now() - timedelta(days = 14): raise AWSQueryStartDateTooOld('When using the HOURLY Granularity. The StartTime can be no older than 14 days.')

In [4]:
def getServiceCosts(client,
                    StartTime:datetime = datetime.now() - timedelta(days = 1), 
                    EndTime:datetime   = datetime.now(),
                    CostMetric:str     = 'BlendedCost',
                    Granularity:str    = 'DAILY'):       # MONTHLY, DAILY or HOURLY
    
    # Validate StartTime
    validateStartDate(StartTime = StartTime)
    
    # Construct dataframes for charge period records and all combined costs
    cost_df = pd.DataFrame()
    rec_df  = pd.DataFrame()
    
    # The format of the timestamps is different when the Granularity is set to HOURLY
    DATE_FORMATTER = '%Y-%m-%dT%H:%M:%SZ' if Granularity == 'HOURLY' else '%Y-%m-%d'
    
    # Retrieve the data from the AWS API
    cost_data = client.get_cost_and_usage(TimePeriod  = {'Start' : StartTime.strftime(DATE_FORMATTER), 
                                                         'End'   : EndTime.strftime(DATE_FORMATTER)},
                                          Granularity =  Granularity,
                                          Metrics     = [CostMetric],
                                          GroupBy     = [{'Type' : 'DIMENSION', 
                                                          'Key'  : 'SERVICE'}],
                                          )['ResultsByTime']
    
    # Populate the record dataframe with the values of a single charge period
    for period_data in cost_data:
        startDate = pd.to_datetime(period_data['TimePeriod']['Start'])
        endDate   = pd.to_datetime(period_data['TimePeriod']['End'  ])
        for service in period_data['Groups']:
            rec_df['Service'  ] = service['Keys']
            rec_df['StartDate'] = startDate
            rec_df['EndDate'  ] = endDate
            rec_df['Amount'   ] = service['Metrics']['BlendedCost']['Amount']
            rec_df['Unit'     ] = service['Metrics']['BlendedCost']['Unit']
            # Append the record dataframe to the combined costs dataframe
            cost_df = pd.concat([cost_df, rec_df]).drop_duplicates().reset_index(drop=True)

    return cost_df  

In [5]:
def validateProfiles(profiles):
    
    # lists to store the results
    validProfiles   = []
    invalidProfiles = []

    # allow us to visualize the progress
    progressBar = tqdm(range(len(profiles)))
    
    # loop through all profiles configured locally
    for profile in profiles:
        # create a Cost Explorer clien
        ecClient = boto3.Session(profile_name = profile).client('ce')
        # if we can get some costs the profile is valid. Otherwise it is invalid
        try:
            getServiceCosts(ecClient)
            validProfiles.append(str(profile))
        except:
            invalidProfiles.append(str(profile))
        
        # increment the progress
        progressBar.update(1)
        
    # change the bar colour on completion
    progressBar.colour = "green"
    
    # return valid and invalid profiles lists
    return {"valid": validProfiles, "invalid": invalidProfiles}

In [6]:
validProfiles = validateProfiles(awsProfiles)["valid"]

  0%|          | 0/6 [00:00<?, ?it/s]

In [7]:
#validProfiles

In [8]:
#ecClient  = boto3.Session(profile_name = 'training').client('ce')
#getServiceCosts(ecClient)